# Neo4j Graph 2024

- Updated to use the official `neo4j` Python driver and maintain compatibility with `ipycytoscape`.
- The examples in this notebook access the [COVID-19-Net](https://github.com/covid-19-net/covid-19-community) Knowledge Graph.

**CITATION**:  Credit goes to the original author Peter W. Rose (pwrose@ucsd.edu) for his great work

In [ ]:
!pip install -qU neo4j ipycytoscape ipywidgets

In [ ]:
import random
import ipycytoscape
from neo4j import GraphDatabase

## Neo4j Connection Info

In [ ]:
URI = "bolt://132.249.238.185:7687"
AUTH = ("reader", "demo")

## Node and edge styles

### Nodes

In [ ]:
node_centered = {
    'selector': 'node',
    'style': {
        'font-size': '10',
        'label': 'data(name)',
        'height': '60',
        'width': '80',
        'text-max-width': '80',
        'text-wrap': 'wrap',
        'text-valign': 'center',
        'background-color': 'blue',
        'background-opacity': 0.6
    }
}

### Edges

In [ ]:
edge_directed = {
    'selector': 'edge',
    'style': {
        'line-color': '#9dbaea',
        'target-arrow-shape': 'triangle',
        'target-arrow-color': '#9dbaea',
        'curve-style': 'bezier'
    }
}

edge_directed_named = {
    'selector': 'edge',
    'style': {
        'font-size': '8',
        'label': 'data(label)',
        'line-color': '#9dbaea',
        'text-rotation': 'autorotate',
        'target-arrow-shape': 'triangle',
        'target-arrow-color': '#9dbaea',
        'curve-style': 'bezier'
    }
}

edge_undirected = {
    'selector': 'edge',
    'style': {
        'line-color': '#9dbaea'
    }
}

### Node Colors

- Generate a random color palette.

In [ ]:
def random_color_palette(n_colors, seed=3):
    """
    Creates a random color palette of n_colors.
    """
    random.seed(seed)
    return ['#' + ''.join([random.choice('0123456789ABCDEF') for _ in range(6)]) for _ in range(n_colors)]

## Helper Functions

### Query and Convert Results to Subgraph

In [ ]:
def query_to_subgraph(driver, query):
    """
    Executes a query and converts the results into a dictionary format
    compatible with ipycytoscape.
    """
    nodes = {}
    edges = []

    with driver.session() as session:
        result = session.run(query)
        for record in result:
            path = record["p"]  # Assumes the query returns a path 'p'
            for node in path.nodes:
                if node.element_id not in nodes:
                    tooltip_data = ",\n".join(f"{key}: {value}" for key, value in node.items())
                    nodes[node.element_id] = {
                        "data": {
                            "id": node.element_id,
                            "name": node.get("name", "Unnamed Node"),
                            "label": list(node.labels)[0] if node.labels else "Unknown",
                            "tooltip": tooltip_data,
                        }
                    }
            for rel in path.relationships:
                tooltip_data = ",\n".join(f"{key}: {value}" for key, value in rel.items())
                edges.append({
                    "data": {
                        "source": rel.start_node.element_id,
                        "target": rel.end_node.element_id,
                        "label": rel.type,
                        "tooltip": tooltip_data,
                    }
                })

    return {"nodes": list(nodes.values()), "edges": edges}

### Fetch Neo4j Data and Visualize

In [ ]:
def fetch_and_visualize(query, layout="dagre", edge_style=edge_directed, node_colors_seed=3):
    """
    Executes a Neo4j query, converts the results to a subgraph, and visualizes it with ipycytoscape.
    """
    driver = GraphDatabase.driver(URI, auth=AUTH)
    try:
        subgraph = query_to_subgraph(driver, query)

        widget = ipycytoscape.CytoscapeWidget()
        widget.graph.add_graph_from_json(subgraph)

        # Set styles
        style = [node_centered, edge_style]

        # Extract unique labels for node-specific colors
        labels = list({node["data"]["label"] for node in subgraph["nodes"]})
        labels.sort()

        colors = random_color_palette(len(labels), seed=node_colors_seed)

        for label, color in zip(labels, colors):
            style.append({'selector': f'node[label = "{label}"]', 'style': {'background-color': color}})

        widget.set_style(style)
        widget.set_layout(name=layout, padding=0)
        widget.set_tooltip_source('tooltip')  # Set the tooltip source

        return widget
    finally:
        driver.close()

## Examples

### Example 1: Cities with the Name "San Francisco"

In [ ]:
query1 = """
MATCH p=(:City{name:'San Francisco'})-[:IN*]->(:World) RETURN p
"""
widget1 = fetch_and_visualize(query1, layout="dagre", edge_style=edge_directed)

In [ ]:
widget1

### Example 2: Proteins Interacting with SARS-CoV-2 Spike Protein

In [ ]:
query2 = """
MATCH p=(:Protein{name: 'Spike glycoprotein', taxonomyId: 'taxonomy:2697049'})-[:INTERACTS_WITH]-(:Protein) RETURN p
"""
widget2 = fetch_and_visualize(query2, layout="concentric", edge_style=edge_undirected)

In [ ]:
widget2

### Example 3: Explore the Data Sources

In [ ]:
query3 = """
MATCH p=(:MetaNode)-[:ETL_FROM]->(:DataSource) RETURN p
"""
widget3 = fetch_and_visualize(query3, layout="klay", edge_style=edge_directed)

In [ ]:
widget3

### Example 4: Metagraph of Nodes and Relationships

In [ ]:
query4 = """
MATCH p=(a:MetaNode)-[:META_RELATIONSHIP]->(b:MetaNode)
WHERE a.name <> 'Location' AND b.name <> 'Location'
RETURN p
"""
widget4 = fetch_and_visualize(
    query4,
    layout="cola",
    edge_style=edge_directed_named,
    node_colors_seed=42
)

In [ ]:
widget4